<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/classification_eval_training.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Welcome to Okareo!



In [ ]:
%pip install okareo transformers torch matplotlib

This notebook trains a model for classifying user queries into "complaints," "returns," or "pricing."

The trained model is used in the Classification Evaluation script:

<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/classification_eval.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

The model is trained on data generated by an Okareo generator. Details on how the data was generated can be found in this notebook:

<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/generating_classification_scenarios.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# Import pandas for data processing
import pandas as pd

# Load data from Okareo's GitHub and convert the JSONL string to a pandas DataFrame
data = pd.read_json(path_or_buf="https://raw.githubusercontent.com/okareo-ai/okareo-python-sdk/main/examples/webbizz_classification_questions.jsonl", lines=True)

# Convert the "result" column to numeric classes
data["label"] = data["result"].map({"complaints": 2, "returns": 1, "pricing": 0})

In [ ]:
# Load a DistilBERT model and tokenizer
from transformers import AutoTokenizer, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=3)

In [ ]:
# Load libraries for PyTorch
import torch
from torch.utils.data import DataLoader

# Split the data into training and validation sets
train_data = data.sample(frac=0.8)
val_data = data.drop(train_data.index)

# Create a custom dataset class for the text data
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the text and label for the given index
        text = self.data.iloc[idx]["input"]
        label = self.data.iloc[idx]["label"]

        # Tokenize the text
        encoding = self.tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

        # Return the input IDs, attention mask, and label (reshape the input IDs and attention mask to remove an unneeded dimension)
        return encoding.input_ids.squeeze(), encoding.attention_mask.squeeze(), label

# Create Dataset objects for the training and validation sets
train_data = TextDataset(train_data, tokenizer)
val_data = TextDataset(val_data, tokenizer)

# Create DataLoader objects for the training and validation sets
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=8)

In [ ]:
# An optimizer for the model
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# One epoch of training
def train(loader, model, optimizer):
    # Set the model to training mode
    model.train()
    print("Training...")
    losses = []
    for input_ids, attention_mask, labels in loader:
        optimizer.zero_grad()
        # One-hot encode the labels
        oh_labels = torch.nn.functional.one_hot(labels, num_classes=3).to(torch.float32)
        # Pass the input IDs, attention mask, and one-hot labels to the model and get the loss
        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=oh_labels).loss
        # Backpropagate the loss
        loss.backward()
        # Update the model parameters
        optimizer.step()
        # Track losses
        losses.append(loss.item())
    return losses

# One epoch of validation
def validate(loader, model, epoch):
    # Set the model to evaluation mode
    model.eval()
    print("Validating...")
    # Disable gradient calculations (not needed for validation)
    with torch.no_grad():
        correct = 0
        total = 0
        for input_ids, attention_mask, labels in loader:
            # Pass the input IDs and attention mask to the model and get the logits
            logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
            # Get the predicted labels
            predictions = torch.argmax(logits, dim=1)
            # Track accuracy
            correct += (predictions == labels).sum().item()
            total += len(labels)
        accuracy = correct / total
        print(f"Epoch {epoch}, Validation Accuracy: {accuracy}")

In [ ]:
# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Train the model for 3 epochs
for epoch in range(3):
    # Train the model for one epoch
    losses = train(train_loader, model, optimizer)
    # Plot the loss
    plt.plot(losses)
    plt.title(f"Epoch {epoch} Loss")
    plt.show()
    # Validate the model
    validate(val_loader, model, epoch)

In [ ]:
# Save the model locally
model.save_pretrained("webbizz_classification_model")

## Pushing to Hugging Face

To make the model publically accessible, Okareo also saved the model on the Hugging Face Hub. After authenticating (not shown), the code below is how the model was published there.

If you train your own model, you can use the local version by omitting `okareo/` (as shown below) when loading the model in your script. 

In [ ]:
# # Load the model from the local directory
# model = DistilBertForSequenceClassification.from_pretrained("webbizz_classification_model")

In [ ]:
# # Save the model to Hugging Face Hub
# model.push_to_hub("webbizz_classification_model")